In [0]:
import os
import openai
import pandas as pd
import re
from matplotlib import pyplot as plt
from wordcloud import WordCloud
from pyspark.sql.functions import *
from pyspark.sql.types import *
from timeout_decorator import timeout
import time
from databricks.feature_store import FeatureStoreClient
from datetime import datetime
from pytz import timezone
from pyspark.sql import Window
from pyspark.sql.functions import row_number
from itertools import chain
import ast

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-3255595830445283>, line 6
      4 import re
      5 from matplotlib import pyplot as plt
----> 6 from wordcloud import WordCloud
      7 from pyspark.sql.functions import *
      8 from pyspark.sql.types import *

ModuleNotFoundError: No module named 'wordcloud'

### Loading Data

In [0]:
df = spark.sql("""select *
               from abfrl_catalog.pfrl_vrdm.rsh_ean_details
               
               limit 10
               """).toPandas()
df

,ProductID,StyleCode,EAN,SIZE,Gender,Category,Brand,ProductName,LongDescription,ShortDescription,ImageURL,ProductLink,a_linkrewrite,EAN_PRICE,HSN_NUMBER,A_ISPUBLISHED,PRODUCT_ACTIVE,a_mpactive,a_sap_material,a_monthandyearofmanufacturer,a_manufacturerinfo,ARTICLECODE,EAN_ACTIVE,a_sap_color,a_categoryid,LiveDate,Accessories,Band_Material,Band_Type,Brand_nm,Brand_Fit,Brand_Fit_Name,Case_Material,Chronograph_1,Collar_1,Collection_1,Color_1,Color_Product_Naming,Cuffs_1,Dial_Color,...,Hood_1,Length_1,Material_1,Mechanism_1,Neck_1,Occasion_1,Pattern_1,Power_Source,Product_Height,Product_Length,Product_Type,Product_Width,Season_1,Shoe_Type,Shorts_Type,Size_1,Sleeves_1,Sole_Material,Story_1,Strap_Color,Style_1,Sub_brand,Suit_Front,Swatch_Color,Trend_1,Trouser_Front,Upper_Material,USP_1,Wash_1,product_care,Fabric_Type,PRIMARY_COLOR,SubCategory,Waist_Rise,Dispatch_by_Days,DATA_SRC_CD,REC_STRT_DT,REC_END_DT,ETL_JOB_NM,modified_ts
0,31550,TG301182649003,205000004480681,3-4Y,Girls,"Tops, Tees &amp; Blouses",Pantaloons,Black Polka Dotted Shirt,<p> Channel some retro fashion into your littl...,<p> Channel some retro fashion into your littl...,None,https://pantaloons.com/p/black-all-over-print-...,black-all-over-print-regular-fit-shirt,499.00000,61012000,0,1,1,None,None,"Aditya Birla Fashion and Retail Limited,Divyas...",None,1,Multi,231,2015-04-18 15:06:46,None,None,None,Pantaloons,None,Regular Fit,None,None,Regular Collar,None,Multi-coloured,None,None,None,...,None,None,Viscose Rayon Blend,None,Regular Collar,Casual,All-over print,None,None,None,None,None,1402,None,None,None,Short Sleeves,None,None,None,None,Chalk,None,None,Fashion,None,None,None,None,None,None,Multi-coloured,None,None,None,1000,2024-08-22,None,RSH PRODUCT DETAILS,2024-08-21 21:52:53.212
1,49016,TK301222883002,205000004504235,M,Women,Ethnic Sets,Pantaloons,Red Embroidered Slim Fit Kurta Churidar Dupatt...,</br></br><b>Model Measurement:</b></br><b>Hei...,</br></br><b>Model Measurement:</b></br><b>Hei...,None,https://pantaloons.com/p/red-embroidered-slim-...,red-embroidered-slim-fit-kurta-churidar-dupatt...,2799.00000,61012000,0,1,1,None,None,"Aditya Birla Fashion and Retail Limited,Divyas...",None,1,Red,217,2015-04-14 14:58:28,None,None,None,Pantaloons,None,Slim Fit,None,None,None,None,Red,None,None,None,...,None,Long Kurta,100% Cotton,None,Round Neck,Ethnic,Embroidered,None,None,None,None,None,1501,None,None,None,Sleeveless,None,None,None,None,Trishaa,None,None,Fashion,None,None,None,None,None,None,Red,None,None,None,1000,2024-08-22,None,RSH PRODUCT DETAILS,2024-08-21 21:52:53.212
2,2645,TO110001068013,205000004626827,XXL,Men,Sweatshirts,Pantaloons,Grey Solid Leisure Sport Full Sleeves Men Regu...,This grey melange knit jacket from JM Sport by...,This grey melange knit jacket from JM Sport by...,None,https://pantaloons.com/p/pantaloons-grey-sweat...,pantaloons-grey-sweatshirt,1499.00000,61012000,0,1,1,None,None,"Aditya Birla Fashion and Retail Limited,Divyas...",None,1,Grey,193,2015-09-22 06:54:36,None,None,None,Pantaloons,None,None,None,None,None,None,Grey,Grey,None,None,...,Hooded,None,60% Cotton and 40% Polyester,None,None,Leisure Sport,Solid,None,None,None,Sweatshirt,None,1502,None,None,None,Full Sleeves,None,None,None,None,JM Sport,None,#ffffff,None,None,None,None,None,None,None,Medium Grey,None,None,None,1000,2024-08-22,None,RSH PRODUCT DETAILS,2024-08-21 21:52:53.212
3,12514,TE110002420008,205000004723752,9-10Y,Boys,Winter wear,Pantaloons,Pantaloons Yellow Sweater,Keep your little boy snug on chilly days with ...,Keep your little boy snug on chilly days with ...,None,https://pantaloons.com/p/pantaloons-yellow-swe...,pantaloons-yellow-sweater,699.00000,61012000,0,1,1,None,None,"Aditya Birla Fashion and Retail Limited,Divyas...",None,1,Yellow,230,2015-09-28 07:07:33,None,None,None,Pantaloons,None,None,None,None,None,None,Yellow,Yellow,None,None,...,None,None,100% Cotton,None,V Neck,Casual,Patterned,None,None,None,Sweater,None,1502,None,None,None,Sleeveless,None,None,N

In [0]:
df = spark.sql("""select ProductName, ShortDescription, LongDescription
               from abfrl_catalog.pfrl_vrdm.rsh_ean_details
               where EAN = '205000005725382'
               --limit 10
               """).toPandas()
df.display()

ProductName,ShortDescription,LongDescription
Red Checked Casual Full Sleeves Shirt Collar Men Slim Fit Casual Shirts,"Red checked casual shirt with a spread collar, long sleeves and a full button placket. Made from cotton in a slim fit, it features a chest pocket.","Back to basics, this red checked casual shirt is the perfect pick of the season. You can team it with a pair of blue washed jeans and canvas lace-ups when going out for a late night movie."


In [0]:
df.display()

modified_ts,DATA_SRC_CD,ETL_JOB_NM,REC_DEL_FLAG,REC_END_DT,REC_STRT_DT,a_all_image_link,a_brand,a_brandfit,a_brandid,a_catalogfeedid,a_category,a_categoryid,a_color,a_createddate,a_currency,a_discount,a_discountpercentage,a_ean,a_gender,a_googlecategory,a_googleproducttype,a_hsn_number,a_image_link,a_livedate,a_material,a_mp_status,a_mrp,a_offlinestock,a_onlinestock,a_product_createddate,a_product_modifieddate,a_productid,a_productlink,a_producttitle,a_producttype,a_reference,a_safetyfactor,a_season,a_sellingprice,a_size,a_status,a_stylecode,a_subbrand,a_taxonomyid,_rescued_data,event_name,__Timestamp,__DeleteVersion,__UpsertVersion,__DROP_EXPECTATIONS_COL,__MEETS_DROP_EXPECTATIONS,__ALLOW_EXPECTATIONS_COL
2024-08-09T03:05:41.692Z,3300,CATALOGFEED_DBX,N,null,2024-08-09,https://assets.abfrlcdn.com/img/app/product/2/209692-656306.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/209692-656307.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/209692-656308.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/209692-656309.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/209692-656311.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/209692-656310.jpg;,Pantaloons,null,8,226576,Tops,10998,Navy,2024-08-08T17:32:27Z,INR,0.00000,0.00000,205000005764504,Women,Apparel & Accessories > Clothing > Shirts & Tops,Tops,61012000,https://assets.abfrlcdn.com/img/app/product/2/209692-656306.jpg?w=1000&h=1000,2017-05-09T07:11:30Z,null,Live,899.00000,0,0,2017-05-09T05:41:32Z,null,209692,https://pantaloons.com/p/akkriti-navy-top-209692.html,Akkriti Navy Cut & Sew Ethnic 3/4th Sleeves Round Neck Women Regular Fit Top,Tops,110028401001_TB110028401324,0,1701,899.00000,M,Live,TB110028401324,Akkriti,0,null,update,1723152981470,null,2024-08-09T03:05:41.692Z,Map(),true,Map()
2024-08-09T03:05:41.692Z,3300,CATALOGFEED_DBX,N,null,2024-08-09,https://assets.abfrlcdn.com/img/app/product/2/264544-1042895.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/264544-1042896.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/264544-1042897.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/264544-1042898.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/264544-1042900.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/264544-1042899.jpg;,Pantaloons,null,8,261747,Churidars,11014,Green,2024-08-08T20:31:15Z,INR,0.00000,0.00000,205000005830632,Women,null,Churidars,61012000,https://assets.abfrlcdn.com/img/app/product/2/264544-1042895.jpg?w=1000&h=1000,2018-02-19T06:37:20Z,null,Live,399.00000,0,0,2018-02-19T04:22:28Z,null,264544,https://pantaloons.com/p/green-solid-casual-women-regular-fit-churidar-264544.html,Rangmanch Green Solid Casual Women Regular Fit Churidar,Churidars,110035406036_TJ110035406256,0,1801,399.00000,2XL,Live,TJ110035406256,Rangmanch,0,null,update,1723152981470,null,2024-08-09T03:05:41.692Z,Map(),true,Map()
2024-08-09T03:05:41.692Z,3300,CATALOGFEED_DBX,N,null,2024-08-09,https://assets.abfrlcdn.com/img/app/product/2/267393-1060730.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/267393-1060731.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/267393-1060732.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/267393-1060733.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/267393-1060735.jpg?w=1000&h=1000;https://assets.abfrlcdn.com/img/app/product/2/267393-1060734.jpg;,Pantaloons,null,8,270912,Pants,11012,Black,2024-08-08T17:31:51Z,INR,0.00000,0.00000,205000005566069,Women,Apparel & Accessories > Clothing > Pants,Pants,61012000,https://assets.abfrlcdn.com/img/app/product/2/267393-1060730.jpg?w=1000&h=1000,2018-03-07T09:34:30Z,null,Live,799.00000,0,0,2018-03-07T08:09:25Z,null,267393,https://pantaloons.com/p/rangmanch-black-pants-267393.html,Rangmanch Black Solid Casual Women Regular Fit Pants,Pants,110008005005_TJ11000800520,0,1601,799.00000,XL,Live,TJ11000800520,Rangmanch

In [0]:
%sql
select *
from abfrl_catalog.pfrl_vrdm.t_pantaloon_materialmaster
where a_retekclass <> ''
limit 10

a_accessories,a_activeweartype,a_bagtype,a_bottomtype,a_brand,a_brandfitname,a_category,a_collar,a_collection,a_color,a_colorproductnaming,a_commodity,a_countyoforigin,a_createdby,a_createddate,a_cuffs,a_eannumber,a_ethnicbottomstyle,a_fabrictype,a_fashionaccessories,a_fit,a_frontopening,a_gender,a_hood,a_hsn_number,a_isvalid,a_kurtastyle,a_length,a_longdesc,a_manufacturerinfo,a_material,a_modifiedby,a_modifieddate,a_monthyear,a_mrp,a_neck,a_netquantity,a_occasion,a_pattern,a_pkid,a_prefix,a_processed,a_productheight,a_productid,a_productlength,a_productmaterial,a_producttype,a_productwidth,a_retekclass,a_season,a_shoetype,a_shoot_material,a_shortdesc,a_shortstype,a_shoulder,a_size,a_sleeves,a_sourcetype,a_straps,a_style,a_subbrand,a_suitfront,a_suittype,a_swatchImageLink,a_taxclassificationonart,a_trend,a_trendin_material,a_trouserfront,a_waistbandtype,a_wash,all_image_link,modified_ts
NULL,NULL,NULL,NULL,Pantaloons,NULL,Bottoms,NULL,NULL,Navy,Navy,null,null,null,null,NULL,8905172268239,NULL,NULL,NULL,Regular Fit,NULL,Boys,NULL,0,1,NULL,Full Length,"Give your kid a cool look with this pair of trousers from Pantaloons Baby. Featuring a solid colour, full length, pockets and detachable suspenders. It is crafted from high-quality cotton fabric providing comfort all day. This pair of trousers would be a trendy inclusion.",,110095047001,23,2021-02-09T12:21:27Z,null,599,NULL,0,Casual,Solid,318617,KB,0,NULL,576459,0,Cotton,Trousers,0,NULL,2021,NULL,110095047001,"Style this pair of trousers with a shirt, socks and casual shoes to complete the comfy outfit.",NULL,NULL,9-12 M,NULL,0,NULL,NULL,Pantaloons Baby,NULL,NULL,https://content.pantaloons.com/PTcom/110095047004_4.jpg,0,Fashion,KB110095047324,NULL,NULL,NULL,https://content.pantaloons.com/PTcom/110095047004_1.jpg;https://content.pantaloons.com/PTcom/110095047004_2.jpg;https://content.pantaloons.com/PTcom/110095047004_3.jpg;https://content.pantaloons.com/PTcom/110095047004_4.jpg,2024-02-09T11:25:58.161Z
NULL,NULL,NULL,NULL,Pantaloons,NULL,Sweaters & Sweatshirts,NULL,NULL,White,White,null,null,null,null,NULL,8905172256489,NULL,NULL,NULL,Regular Fit,NULL,Men,NULL,0,1,NULL,NULL,"Build in your charisma with this sweatshirt from Byford. It features a graphic print, round neck and full sleeves. Crafted from cotton polyester blend offering utmost comfort. This sweatshirt will be a sport addition to your wardrobe.",,110094871001,23,2021-02-09T12:21:27Z,null,899,Round Neck,0,Casual,Print,318618,BY,0,NULL,576249,0,Cotton Polyester,Sweatshirt,0,NULL,2021,NULL,110094871001,"Pair this sweatshirt with jeans, sneakers and a backpack to complete the casual outfit.",NULL,NULL,XXL,Full Sleeves,0,NULL,NULL,Byford,NULL,NULL,https://content.pantaloons.com/PTcom/110094871002_6.jpg,0,Fashion,BY110094871471,NULL,NULL,NULL,https://content.pantaloons.com/PTcom/110094871002_1.jpg;https://content.pantaloons.com/PTcom/110094871002_2.jpg;https://content.pantaloons.com/PTcom/110094871002_3.jpg;https://content.pantaloons.com/PTcom/110094871002_4.jpg;https://content.pantaloons.com/PTcom/110094871002_5.jpg,2024-02-09T11:25:58.161Z
NULL,NULL,NULL,NULL,Pantaloons,NULL,Sweaters & Sweatshirts,NULL,NULL,White,White,null,null,null,null,NULL,8905172256496,NULL,NULL,NULL,Regular Fit,NULL,Men,NULL,0,1,NULL,NULL,"Build in your charisma with this sweatshirt from Byford. It features a graphic print, round neck and full sleeves. Crafted from cotton polyester blend offering utmost comfort. This sweatshirt will be a sport addition to your wardrobe.",,110094871001,23,2021-02-09T12:21:27Z,null,899,Round Neck,0,Casual,Print,318619,BY,0,NULL,576249,0,Cotton Polyester,Sweatshirt,0,NULL,2021,NULL,110094871002,"Pair this sweatshirt with jeans, sneakers and a backpack to complete the casual outfit.",NULL,NULL,M,Full Sleeves,0,NULL,NULL,Byford,NULL,NULL,https://content.pantaloons.com/PTcom/110094871002_6.jpg,0,Fashion,BY110094871471,NULL,NULL,NULL,https://content.pantaloons.com/PTcom/110094871002_1.jpg;https://content.pantaloons.com/PTcom/

In [0]:
%sql
select distinct Brand
from abfrl_catalog.pfrl_vrdm.rsh_ean_details


Brand
Jaypore
Shantanu and Nikhil
Pantaloons


In [0]:
%sql
select *
from abfrl_catalog.pfrl_vrdm.rma_material_details
--where AWBNo = 'TG301182649003'
limit 10

modified_ts,AWBNo,BrandOrMp,DATA_SRC_CD,DeliveredToWHDate,ETL_JOB_NM,ItemCount,ItemId,LSPAssignedDate,LastPrevLspAssignedDate,LogisticPartner,ModifiedDate,OrderId,PickedDate,PickedLsp,Pickedawb,PickupCancelReason,REC_END_DT,REC_STRT_DT,Reason,ReturnNode,RmaCreatedDate,RmaId,ShipmentId,Status,StyleCode,prevLSP,_rescued_data,event_name,__Timestamp,__DeleteVersion,__UpsertVersion,__DROP_EXPECTATIONS_COL,__MEETS_DROP_EXPECTATIONS,__ALLOW_EXPECTATIONS_COL
2023-06-12T04:03:28.391Z,9173617908283,Brand.com,1000,2022-10-22T10:56:43Z,tdrmadetails_flamingo,1,8905287841761,2022-10-13T15:32:49Z,null,Delhivery Omni,2022-10-22T11:02:05Z,819491036,2022-10-14T07:18:04Z,Delhivery Omni,9173617908283,null,9999-12-30T18:30:00Z,2023-06-12,null,W555_4007,2022-10-13T15:32:47Z,215327809,25267145,Delivered To WH,Style code,null,null,insert,1686522822789,null,2023-06-12T04:03:28.391Z,Map(),true,Map()
2024-01-09T03:39:07.432Z,R742005979PTN,Brand.com,1000,2024-01-08T12:27:47Z,tdrmadetails_flamingo,1,8905685182794,2024-01-01T09:41:02Z,null,ShadowFax,2024-01-08T12:29:01Z,8240447,2024-01-02T06:31:01Z,ShadowFax,R742005979PTN,null,9999-12-30T18:30:00Z,2024-01-09,null,D018_7500,2024-01-01T09:41:00Z,111903557,36648312,Delivered To WH,Style code,null,null,update,1704751762332,null,2024-01-09T03:39:07.432Z,Map(),true,Map()
2023-06-12T04:03:28.391Z,1321724134271,Brand.com,1000,2022-11-08T06:53:28Z,tdrmadetails_flamingo,1,8905500490752,2022-11-04T17:26:36Z,null,Delhivery,2022-11-08T07:30:24Z,7033789,2022-11-05T09:15:07Z,Delhivery,1321724134271,null,9999-12-30T18:30:00Z,2023-06-12,null,D015_7500,2022-11-04T17:26:34Z,111470078,25913415,Delivered To WH,Style code,null,null,insert,1686522822789,null,2023-06-12T04:03:28.391Z,Map(),true,Map()
2023-12-28T03:40:25.456Z,R732829264PTN,Brand.com,1000,2023-12-27T08:43:16Z,tdrmadetails_flamingo,1,8905685026944,2023-12-20T17:54:54Z,null,ShadowFax,2023-12-27T08:44:01Z,8209147,2023-12-21T13:28:02Z,ShadowFax,R732829264PTN,null,9999-12-30T18:30:00Z,2023-12-28,null,D023_7000,2023-12-20T17:54:53Z,111891121,36346144,Delivered To WH,Style code,null,null,update,1703715040619,null,2023-12-28T03:40:25.456Z,Map(),true,Map()
2023-06-12T04:03:28.391Z,R533647712WEB,Brand.com,1000,null,tdrmadetails_flamingo,1,8905500960019,2023-05-18T11:03:08Z,null,ShadowFax Omni,2023-06-03T21:58:02Z,824216850,null,null,null,null,9999-12-30T18:30:00Z,2023-06-12,null,WBINPT_BH,2023-05-18T11:03:06Z,111636750,31341487,Pickup Cancelled,Style code,null,null,insert,1686522822789,null,2023-06-12T04:03:28.391Z,Map(),true,Map()
2023-06-12T04:03:28.391Z,R399543039TDN,Brand.com,1000,2022-11-30T10:52:49Z,tdrmadetails_flamingo,1,8905652126622,2022-11-24T05:19:35Z,null,ShadowFax,2022-11-30T11:02:01Z,7104616,2022-11-24T13:15:02Z,ShadowFax,R399543039TDN,null,9999-12-30T18:30:00Z,2023-06-12,null,W508_4007,2022-11-24T05:19:34Z,215477481,26363591,Delivered To WH,Style code,null,null,insert,1686522822789,null,2023-06-12T04:03:28.391Z,Map(),true,Map()
2023-06-12T04:03:28.391Z,R393467752WEB,Brand.com,1000,null,tdrmadetails_flamingo,1,8905172750871,2022-11-15T08:23:44Z,null,ShadowFax Omni,2022-12-01T23:02:07Z,820085529,null,null,null,null,9999-12-30T18:30:00Z,2023-06-12,null,WBINPT_BH,2022-11-15T08:23:42Z,111477816,26100870,Pickup Cancelled,Style code,null,null,insert,1686522822789,null,2023-06-12T04:03:28.391Z,Map(),true,Map()
2023-06-12T04:03:28.391Z,R520954372WEB,Brand.com,1000,2023-05-10T18:09:27Z,tdrmadetails_flamingo,1,8905500050505,2023-05-03T06:48:21Z,null,ShadowFax Omni,2023-05-10T18:10:03Z,823933987,2023-05-04T06:33:02Z,ShadowFax Omni,R520954372WEB,null,9999-12-30T18:30:00Z,2023-06-12,null,WBINPT_BH,2023-05-03T06:48:19Z,111613373,30994540,Delivered To WH,Style code,null,null,insert,1686522822789,null,2023-06-12T04:03:28.391Z,Map(),true,Map()
2023-09-21T03:07:48.915Z,R639874883TDN,Brand.com,1000,2023-09-20T10:38:33Z,tdrmadetails_flamingo,1,8905678134427,2023-09-12T17:00:41Z,null,ShadowFax,2023-09-20T10:39:02Z,SA180605772,2023-09-13T10:18:01Z,ShadowFax,R639874883TDN,null,9999-

In [0]:
%sql
select *
from abfrl_catalog.ml_tables.gen_ai_product_description
where a_flagged_style = '1'
limit 10

a_stylecode,a_shortdescription,a_Longdescription,a_id,a_productid,a_processed,a_createddate,a_createdby,a_brand,a_flagged_style
23000067316001,Solid Shirt Mini Dresses,"Introducing the light blue mini dress from Forever21, perfect for any casul or workwear occasion. Our solid dress features a trendy collar and full length sleeves, making it fashionable yet practical for any setting. Tie-dye-inspired hues enhance the delicate color while its loose silhouette provides comfortable room for movement. From office days to weekend walks, our mini dress is an excellent choice to stay cool and chic this season. Do yourself a favor and stay up-to-date with the latest fashion trends with our mini dress from Forever21.",,,0,2024-04-09 05:55:43,99,null,1
23000079902010,Regular Fit Graphic Graphic T-Shirts,"Introducing the Forever21 blue graphic t-shirt with a classic collar! Perfect for casual occasions, this shirt is designed for sure-fire style. The graphic print adds a pop of personality to any outfit, while the regular fit keeps it comfortable during any activity. Created with exceptional quality, this shirt from our collection of graphic t-shirts is a must-have for fashion-forward individuals seeking a versatile addition to their wardrobe. Shop now to add a touch of color to your style with Forever21!",,,0,2024-04-09 05:55:43,99,null,1
23000079977005,Solid Sport bras,"Looking for a sport bra that combines functionality and style? Look no further than this solid orange sport bra from the Forever21 collection. With a regular fit and a comfortable round neck, this sleeveless top is designed to cater to all your sport needs, without compromising on style. So whether you're working out in the gym or going for a morning jog, this sport bra has got you covered, literally! So what are you waiting for? Give your exercise routine the refresh it needs with this trendy sport bra today.",,,0,2024-04-09 05:55:43,99,null,1
23000080357005,Solid Bodycon Mini Dresses,"Elevate your party wardrobe with this beige mini dress from Forever21. Designed with a regular fit and solid pattern, this dress exudes elegance that's perfect for special occasions. The full sleeve and round neck add to the dress's comfortable and stylish appeal. Whether it's dancing the night away or catching up with friends, this dress will have you feeling confident and fashion-forward. Each piece of Forever21's collection is designed to make you feel your best self, and this mini dress is no exception.",,,0,2024-04-09 05:55:43,99,null,1
23000080469010,Checked Regular Fit Cropped Crop Tops,"Introducing the latest sleeveless crop top from Forever21, check-patterned in a refreshing blue hue. Durable and comfortable, this regular-fit top features a scoop neckline that will fit any casual or party occasion perfectly. With its elegant design, you can effortlessly make a fashion statement. Mix and match with your favourite high-waisted trousers or skirts and create your own style. Grab yours now to elevate your closet to the next level with a must-have edition from Forever21.",,,0,2024-04-09 05:55:43,99,null,1
23000080561011,Regular Fit Solid Solid T-Shirts,"Make a bold style statement with the Forever21 blue solid round neck t-shirt. Designed to fit comfortably, this regular fit t-shirt is perfect for a casual outing. The short sleeves add breathability to provide all day comfort. This t-shirt is sure to add distinctiveness to your wardrobe, a perfect piece that exudes calm and serenity. Get ready to slay your next weekend look with this stylish t-shirt.",,,0,2024-04-09 05:55:43,99,null,1
23000080577015,Solid Shirt Mini Dresses,Stay chic and comfortable with this solid cream mini dress by Forever21. Its half-sleeve design makes it perfect for a casual occasion- elegant enough to make you stand out without grabbing too much attention. The soft collar and solid pattern add a touch of sophistication to the classic dress. It can easily be paired with accessories to suit your style. This dress 

In [0]:
%sql
select distinct(productid), stylecode, sub_brand, 
               gender, Material_1, product_type, fit_1, waist_rise, 
               pattern_1 ,color_1, wash_1, Occasion_1, ProductLink 
from abfrl_catalog.pfrl_vrdm.rsh_ean_details
limit 10
--where brand='Pantaloons'

productid,stylecode,sub_brand,gender,Material_1,product_type,fit_1,waist_rise,pattern_1,color_1,wash_1,Occasion_1,ProductLink
776446,COO110123430260,Coolsters,Boys,Cotton,Shirts,Regular Fit,null,Printed,Light Blue,null,Casual,https://pantaloons.com/p/light-blue-shirt-776446.html
905029,UCB330497760399,UCB,Men,Cotton,T-Shirts,Relaxed Fit,null,Solid,Red,null,Casual,https://pantaloons.com/p/red-solid-casual-short-sleeves-round-neck-men-relaxed-fit-t-shirt-905029.html
913228,DZ110140895345,Dreamz,Women,Cotton,Tops,Relaxed Fit,null,Printed,Olive,null,Sleepwear,https://pantaloons.com/p/olive-print-sleepwear-women-comfort-fit-top-913228.html
878136,EE330454778211,Idee,Men,Metal,Sunglasses,null,null,null,Multi-coloured,null,Casual,https://pantaloons.com/p/multi-coloured-sunglasses-878136.html
915569,BY110141284213,Byford,Men,Cotton,T-Shirts,Slim Fit,null,Textured,Medium Grey,null,Casual,https://pantaloons.com/p/medium-grey-stripe-casual-half-sleeves-polo-neck-men-slim-fit-t-shirt-915569.html
568973,KB110095089324,Pantaloons Baby,null,Cotton,T-Shirts,Regular Fit,null,Textured,Navy,null,Casual,https://pantaloons.com/p/navy-stripeds-t-shirt-568973.html
612236,TF11009530729,Bare Denim,Men,Cotton,Jeans,Skinny Fit,null,Solid,Blue,null,Casual,https://pantaloons.com/p/blue-tint-jeans-612236.html
519928,PEO110068742368,People,Women,Rayon,Tops,Regular Fit,null,Textured,Pink,null,Casual,https://pantaloons.com/p/people-pink-textured-top-519928.html
788322,KJ110130379320,Pantaloons Junior,Girls,Cotton,T-Shirts,Regular Fit,null,Printed,Multi-coloured,null,Casual,https://pantaloons.com/p/multicoloured-t-shirts-pack-of-2-788322.html
556207,ZL330356831368,Zink London,Women,Polyester,Dresses,Regular Fit,null,Solid,Pink,null,Casual,https://pantaloons.com/p/zink-london-pink-solid-dress-556207.html
